# Lokaverkefni: Leiðarval hjá Strætó

In [ ]:
from posixpath import split

def convertTime(t1, t2):
  t1_hours = int(t1.split(':')[0])
  t1_min = int(t1.split(':')[1])
  t1_seconds = int(t1.split(':')[2])

  t2_hours = int(t2.split(':')[0])
  t2_min = int(t2.split(':')[1])
  t2_seconds = int(t2.split(':')[2])


  t1_minutes = (t1_hours * 60 ) + t1_min + (t1_seconds / 60)
  t2_minutes = (t2_hours * 60 ) + t2_min + (t2_seconds / 60)

  diff = round((max(t1_minutes, t2_minutes) - min(t1_minutes, t2_minutes)), 2)

  return diff


t1 = '09:30:20'
t2 = '09:10:00'
print(convertTime(t1, t2))

In [ ]:
def time_to_minutes(ti):
  splitted = ti.split(':')
  
  hours = int(splitted[0])
  min = int(splitted[1])
  seconds = int(splitted[2])

  return (hours * 60) + min + (seconds / 60)

In [ ]:
# Graph stuff

def add_vertex(g, v):
  if v in g:
    raise Exception("Hnúturinn er þegar í netinu")
  else:
    g[v] = []
  return g

def add_edge(g, fro, to, data):
  if fro not in g:
    raise Exception(fro + " er ekki í netinu")
  if to not in g:
    raise Exception(to + " er ekki í netinu")
  g[fro].append([data, to])
  return g

def get_edges(g, v):
  return g[v]

In [ ]:
import csv
import sys

#0: stop_id
#1: stop_name
#2: stop_lat
#3: stop_lon
#4: location_type
stoppistodvar = []
with open(sys.path[0]+'/gtfs/stops.txt', newline='', encoding="utf-8") as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    next(spamreader, None) # skip the header
    for row in spamreader:
        stoppistodvar.append(row)

#0: trip_id
#1: arrival_time
#2: departure_time
#3: stop_id
#4: stop_sequence
#5: stop_headsign
#6: pickup_type
leidir = []
with open(sys.path[0]+'/gtfs/stop_times.txt', newline='', encoding="utf-8") as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    next(spamreader, None) # skip the header
    paths = {}
    for row in spamreader:
      if row[6] == '0':
        if row[0] not in paths:
          paths[row[0]] = []
        paths[row[0]].append(row)

# An example of a path
for x in paths['538402']:
  print(x)

for x in stoppistodvar[:50]:
  print(x)

stoppistodvar_id_to_name = {}
for x in stoppistodvar:
    stoppistodvar_id_to_name[x[0]] = x[1]

In [ ]:
# Making the graph

graph = {}

for x in stoppistodvar:
  graph = add_vertex(graph, x[0])

for path in paths:
  for s in range(len(paths[path])-1):
    interval_name = 'p' + path + '_' + paths[path][s][4]
    graph = add_vertex(graph, interval_name)
    graph = add_edge(graph, paths[path][s][3], interval_name, paths[path][s][2])
    graph = add_edge(graph, interval_name, paths[path][s+1][3], paths[path][s+1][1])

print(len(graph))

In [ ]:
# g: graph
# x: origin
# y: destination
# t: beginning time

def find_path(g, x, y, t):
    t = time_to_minutes(t)
    if x not in g:
        raise Exception(x + " er ekki í netinu")
    unvisited_nodes = list(g.keys())
    earliest = {}
    previous_nodes = {}
    for node in unvisited_nodes:
        earliest[node] = sys.maxsize
    earliest[x] = t
    while unvisited_nodes: # Need to find an algorithm that isn't O(n^2)
        current_min_node = None
        for node in unvisited_nodes:
            if current_min_node == None:
                current_min_node = node
            elif earliest[node] < earliest[current_min_node]:
                current_min_node = node
        if earliest[current_min_node] == sys.maxsize:
            break
        e = get_edges(g, current_min_node)
        if current_min_node == y:
            break
        for edge in e:
            if earliest[edge[1]] >= time_to_minutes(edge[0]) and time_to_minutes(edge[0]) >= earliest[current_min_node]:
                earliest[edge[1]] = time_to_minutes(edge[0])
                previous_nodes[edge[1]] = current_min_node
        unvisited_nodes.remove(current_min_node)
    # Change the following for better formatting
    bus_trip = y
    print(stoppistodvar_id_to_name[y], minutes_to_time(earliest[y]))
    while bus_trip != x:
        bus_trip = previous_nodes[bus_trip]
        bus_trip = previous_nodes[bus_trip]
        print(stoppistodvar_id_to_name[bus_trip], minutes_to_time(earliest[bus_trip]))

find_path(graph, '90000097', '90000446', "11:30:00")

In [ ]:
def minutes_to_time(t):
    t = int(t)
    hours = t // 60
    minutes = t % 60
    return f'{hours:02d}:{minutes:02d}'

print(minutes_to_time(780))